In [2]:
import sqlite3

# Connect to SQLite database
connection = sqlite3.connect('Digital_Tourism.db')
connection.row_factory = sqlite3.Row
cursor = connection.cursor()

cols = "Name, Email"
check_record_query = f'''
SELECT {cols} FROM user_inputs WHERE Phone_num = ? LIMIT 1;
'''
cursor.execute(check_record_query, ("94769818269",))

rows = cursor.fetchall()
records = [dict(row) for row in rows]

connection.commit()
connection.close()

print(records)

[{'Name': 'Gowshalini', 'Email': 'g@g.com'}]


In [3]:
#retrieve_information

from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
import json 

from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

user_input="I'm interested in visiting beaches and we are 10 people"
received_phone_num = "942455456"
load_dotenv()

GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
classes =["Question", "other"]
# Initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_tokens=250,
    temperature=0.5,
)

    # Create prompt template
prompt = ChatPromptTemplate.from_messages(
        [
            ( "system",
                f"""
You are a information retrieval ChatBot. Retrieve the following information {', '.join(classes)} from the user_input.

Description for information are:
- 'Name refers to the name of the user.',
- 'Email refers to the email of the user and the input should be in the default email format.',
- 'Address refers to the address of the user.',
- 'num_of_days refers to how many days users plan to spend on their holiday or trip, and the input should be in numbers.',
- 'num_of_adults refers to how many members plan to stay in a hotel or room, and the input should be in numbers.',
- 'month refers to which month users plan to enjoy their trip or holiday, and the input should be the name of the month.',
- 'Start_date refers to the day the trip starts, and the input should be in the date format.',
- 'End_date refers to the day the trip ends, and the input should be in the date format.',
- 'Interests refers to the user's interests or preferences for the trip (e.g., beach, wildlife, nature, adventure).'

Current Question:
{user_input}

example for output:
UPDATE user_inputs
SET Name = "John", Email = "john@gmail.com", Address = "Swiss", Num_of_days = "7", Num_of_adults = "10", Month = "june", Start_date = "2024/10/10", End_date = "2024/10/20", Interests = "nature and beaches"
WHERE Phone_num = {received_phone_num};

Instructions:
- Please Check weather the current question into one of the above information.
- The output should be a sqlite3 update query with retrieved information.
- in the output only change the line starts with SET.
- the output should be strictly in given format(sql query only) and avoid unnecessary words.
- If the keys in line SET doesn't have values please remove those keys.
- If all of the keys doesn't have values just output SELECT 1 FROM user_inputs;
"""
                    
            ),
            ("human", "{user_input}"),
        ]
    )

# Create chain
chain = prompt | llm

# Generate the answer
ai_msg = chain.invoke({
    "user_input": user_input
})

# Retrieve the results
result = ai_msg.content
print(result)

I0000 00:00:1722836222.526337   29088 tcp_posix.cc:809] IOMGR endpoint shutdown


UPDATE user_inputs
SET Interests = "beaches", Num_of_adults = "10"
WHERE Phone_num = 942455456;



I0000 00:00:1722836465.057657   29100 tcp_posix.cc:809] IOMGR endpoint shutdown


In [19]:
#Analyse intent
# Define the classes for intent analysis
text = "How are u"
classes = ["Question", "Not Question"]

print("WELCOME TO INTENT ANALYSIS SECTION")

# Initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_tokens=250,
    temperature=0.5,
)

# Create prompt template
prompt = ChatPromptTemplate.from_messages(
        [
            ( "system",
                f"""
You are a intent classification bot. Classify the text into one of the following categories: {', '.join(classes)}

Current text:
{text}

Instructions:
- The {', '.join(classes)} are in classification priority order.
- output should be the category.
"""
            ),
            ("human", "{text}"),
        ]
    )

# Create chain
chain = prompt | llm

# Generate the answer
ai_msg = chain.invoke({
    "text": text
})

# Retrieve the results
result = ai_msg.content

print(result)

WELCOME TO INTENT ANALYSIS SECTION


I0000 00:00:1722581128.102191  477915 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1722581128.250282  477903 tcp_posix.cc:809] IOMGR endpoint shutdown


Question 



I0000 00:00:1722581354.365284  477915 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1722581370.117963  477915 tcp_posix.cc:809] IOMGR endpoint shutdown


In [ ]:
            You are a ChatBot. Your task is to generate questions to collect the missing values in the information. The information is in the form of json object. 
            The missing values in the information are filled with ''.
            
            The following are the description for the keys in the information collected.
            - 'Name refers to the name of the user.',
            - 'Email refers to the email of the user and the input should be in the default email format.',
            - 'Address refers to the address of the user.',
            - 'num_of_days refers to how many days users plan to spend on their holiday or trip, and the input should be in numbers.',
            - 'num_of_adults refers to how many members plan to stay in a hotel or room, and the input should be in numbers.',
            - 'month refers to which month users plan to enjoy their trip or holiday, and the input should be the name of the month.',
            - 'start_date refers to the day the trip starts, and the input should be in the date format.',
            - 'end_date refers to the day the trip ends, and the input should be in the date format.',
            - 'interests refers to the user's interests or preferences for the trip (e.g., beach, wildlife, nature, adventure).'

            Sample questions to generate:
            - 'Welcome to Sri Lanka Travel Assistant! How many days are you planning for your tour?' (num_of_days)
            - 'How many people will be staying?' (num_of_adults)
            - 'Which month will you be visiting Sri Lanka?' (month)
            - 'What activities or sights are you interested in? (You can choose multiple: Beaches, Wildlife, Culture, Mountains, Historical Places, Scenic Train Ride)' (interests)

            instructions:
            - Generate one question based on the missing values in information. 
            - Don't generate questions if values are present. 
            - Filling Address, month, start_date, end_date, interests are optional. 

In [20]:
import sqlite3
import json

def get_chat_records(received_phone_num, columns):
	# Connect to SQLite database
	connection = sqlite3.connect('Digital_Tourism.db')
	connection.row_factory = sqlite3.Row
	cursor = connection.cursor()

	check_record_query = f'''
	SELECT {columns} FROM message_store WHERE session_id = ?;
	'''
	cursor.execute(check_record_query, (received_phone_num,))

	rows = cursor.fetchall()
	records = [dict(row) for row in rows]
	
	connection.commit()
	connection.close()

	return records

# get prev question record
cols = "message"
table = "message_store"
records = get_chat_records("94769818269", cols )
print(records)
message = json.loads(records[-1]["message"])
message["data"]["content"]

#get chat history
# Extract the 'type' and 'content' fields
extracted_data = []

for item in records:
    message_data = json.loads(item['message'])
    message_type = message_data.get('type')
    message_content = message_data.get('data', {}).get('content')
    
    if message_type and message_content:
        extracted_data.append({
            'type': message_type,
            'content': message_content
        })

print(extracted_data)
# # Display extracted data
# for message in extracted_data:
#     print(f"Type: {message['type']}, Content: {message['content']}")


[{'message': '{"type": "human", "data": {"content": "Clear", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null, "example": false}}'}, {'message': '{"type": "ai", "data": {"content": "The database is cleared. How can I help you?", "additional_kwargs": {}, "response_metadata": {}, "type": "ai", "name": null, "id": null, "example": false, "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": null}}'}, {'message': '{"type": "human", "data": {"content": "Hi", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null, "example": false}}'}, {'message': '{"type": "ai", "data": {"content": "Hi there! How can I assist you today? \\n", "additional_kwargs": {}, "response_metadata": {}, "type": "ai", "name": null, "id": null, "example": false, "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": null}}'}, {'message': '{"type": "human", "data": {"content": "I want to plan a trip", "additional_kwargs": {}, "r

I0000 00:00:1725560071.008491  137047 tcp_posix.cc:809] IOMGR endpoint shutdown


In [5]:
from geminiai_bot_sql import llm_common
from tools import get_chat_history
from SQL_IUD import get_chat_records
import json

received_phone_num = '94769818269'
user_input = "I want to plan a trip"
cols = "message"
table = "message_store"
records = get_chat_records(received_phone_num, cols )
print(records)


chat_history = get_chat_history(records)
chat_history.append({"type": "human", "content": user_input})
print(chat_history)
chat_history_str = json.dumps(chat_history)
print(chat_history_str)

prompt = f"""
Extract values of chat_history only.

*You are an assistant tasked with summarizing chat history. Read through the conversation from chat_history and provide a concise summary that captures the key points, important details, and actions discussed. Focus on the user’s main questions, your responses, and any decisions or agreements made.*

**Chat history Summary**:
"""

history_status = False
keys = {
    "chat_history":chat_history_str,
    "received_phone_num": received_phone_num
    }
response = llm_common(prompt, history_status, keys)
print("summary chat:", response)

[{'message': '{"type": "human", "data": {"content": "Clear", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null, "example": false}}'}, {'message': '{"type": "ai", "data": {"content": "The database is cleared. How can I help you?", "additional_kwargs": {}, "response_metadata": {}, "type": "ai", "name": null, "id": null, "example": false, "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": null}}'}, {'message': '{"type": "human", "data": {"content": "Hi", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null, "example": false}}'}, {'message': '{"type": "ai", "data": {"content": "Now you are in contact with our consultant.", "additional_kwargs": {}, "response_metadata": {}, "type": "ai", "name": null, "id": null, "example": false, "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": null}}'}, {'message': '{"type": "human", "data": {"content": "Hi", "additional_kwargs": {}, "response_metadata"

I0000 00:00:1725562365.991599  157060 tcp_posix.cc:809] IOMGR endpoint shutdown


summary chat: The user initiated a conversation by saying "Hi". The AI responded with a greeting and offered assistance in planning a trip to Sri Lanka. The user then expressed their desire to plan a trip. 



I0000 00:00:1725562607.958175  157069 tcp_posix.cc:809] IOMGR endpoint shutdown


In [31]:
#send a whatsPP MESSAGE
response = "gewf"
url = f"https://graph.facebook.com/{VERSION}/{PHONE_NUMBER_ID}/messages"
headers = {
    f"Authorization": f"Bearer {WHAT_TOKEN}",
    "Content-Type": "application/json"
}
data = {
    "messaging_product": "whatsapp",
    "to": received_phone_num,
    "type": "text",
    "text": {"body": response}
}

response = requests.post(url, json=data, headers=headers)  #send msg to whatsapp


In [92]:
#send a whatsPP pdf
import requests

WHAT_TOKEN = "EAAOXhdmUSwoBO0xsTny64TmlGg5Gb4EyLMovKZCshCeeVtbx3tJPalK1HNiOAzZBkyvCUZArr4N57F1WNklfjMGAar5Fceo1RyJK9GZCLZCCSwdajQNaAlvnZAPtBP7xKQmZBzLxvRGaVeZBKxjZCAxFW3yZCPcl1tg5DsmfhxXwyHycLiZALZBQbcu12i8RpvrgSqCHNISIW0bHDeXU1zTq5ZCXl"
VERSION = "v20.0"
PHONE_NUMBER_ID = "367811079754331"
received_phone_num = "94769818269"
media_id = 892302872719363

# #send by link
# url = f"https://graph.facebook.com/{VERSION}/{PHONE_NUMBER_ID}/messages"
# headers = {
#     f"Authorization": f"Bearer {WHAT_TOKEN}",
#     "Content-Type": "application/json"
# }
# data = {
#     "messaging_product": "whatsapp",
#     "to": received_phone_num,
#     "type": "document",
#     "document": {"link": "https://drive.google.com/file/d/1uxHSpxi9SBIKYDxNv72pyzLhFBOHaPxY/view/Real_Time_Video_Surveillance_for_Automated_Weapon_.pdf", "filename":"student-annotated.pdf"}
# }

# response = requests.post(url, json=data, headers=headers)  #send msg to whatsapp
# response.text


#send file from whatsapp media
url = f"https://graph.facebook.com/{VERSION}/{PHONE_NUMBER_ID}/messages"
medua_id = '1019176409797168'
headers = {
    f"Authorization": f"Bearer {WHAT_TOKEN}",
    "Content-Type": "application/json"
}
data = {
    "messaging_product": "whatsapp",
    "to": received_phone_num,
    "type": "document",
    "document": {          
            "id": media_id,
            "caption": "Here is the PDF document you requested.", 
            "filename": "your-document-filename.pdf"
            }
}

response = requests.post(url, json=data, headers=headers)  #send msg to whatsapp
response.text


'{"messaging_product":"whatsapp","contacts":[{"input":"94769818269","wa_id":"94769818269"}],"messages":[{"id":"wamid.HBgLOTQ3Njk4MTgyNjkVAgARGBJFMzRFNEVEMkI1MkVEQzQwMTYA"}]}'

In [127]:
# CURL
#upload image to whatsapp media using curl
WHAT_TOKEN = "EAAOXhdmUSwoBO0xsTny64TmlGg5Gb4EyLMovKZCshCeeVtbx3tJPalK1HNiOAzZBkyvCUZArr4N57F1WNklfjMGAar5Fceo1RyJK9GZCLZCCSwdajQNaAlvnZAPtBP7xKQmZBzLxvRGaVeZBKxjZCAxFW3yZCPcl1tg5DsmfhxXwyHycLiZALZBQbcu12i8RpvrgSqCHNISIW0bHDeXU1zTq5ZCXl"
VERSION = "v20.0"
PHONE_NUMBER_ID = "367811079754331"
received_phone_num = "94769818269"

!curl -X POST 'https://graph.facebook.com/v20.0/{PHONE_NUMBER_ID}/media' \
-H 'Authorization: Bearer {WHAT_TOKEN}' \
-F 'file=@"00000020-PHOTO-2024-05-14-22-02-35.jpg"' \
-F 'type="image/jpeg"' \
-F 'messaging_product="whatsapp"'



I0000 00:00:1723021495.340932   76391 work_stealing_thread_pool.cc:321] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1723021495.341168   76391 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


{"id":"474184995604037"}

In [13]:
# CURL
#upload pdf to whatsapp media using curl

# server details
WHAT_TOKEN = "EAAWwQCFWwFUBO2xXq51kpurUnhIWS5QAt14F5SVS2ZBu6ZAuHwDicRRDKvjZBWRtoF3l7dDJRZBQg4sRg662WZA9wiKkORRbSpzT1IbQaTkZBsr0BZCoMY45XZCZAwkkejVKSJ97wacZCxbx5RzjZBk9lVMcsd7gZBzH1Vsux6yRw8nVFzeYf6pg3ryFAQaXxXM8839wuvZBQv2kMdweZAq2zdqikZD"
VERSION = "v20.0"
PHONE_NUMBER_ID = "367811079754331"

#LOCAL
#media id  for itinerary pdf= 892302872719363
# !curl -X POST 'https://graph.facebook.com/v20.0/{PHONE_NUMBER_ID}/media' \
# -H 'Authorization: Bearer {WHAT_TOKEN}' \
# -F 'file=@"00000104-Sedan Car Types.pdf"' \
# -F 'type="pdf"' \
# -F 'messaging_product="whatsapp"'

# #media id  for hotel list pdf= 1678828762869228
# !curl -X POST 'https://graph.facebook.com/v20.0/{PHONE_NUMBER_ID}/media' \
# -H 'Authorization: Bearer {WHAT_TOKEN}' \
# -F 'file=@"Hotel_rates_output_pdf/Hotel_rates_output.pdf"' \
# -F 'type="pdf"' \
# -F 'messaging_product="whatsapp"'

#SERVER
#media id  for itinerary pdf
!curl -X POST 'https://graph.facebook.com/v20.0/{PHONE_NUMBER_ID}/media' \
-H 'Authorization: Bearer {WHAT_TOKEN}' \
-F 'file=@"/home/senzmatepc27/Desktop/senzmate/Internal projects/Digital tourism/programs/Programs/Summer Sept-Oct PDF/13  nights _ 14 Days Sri Lanka Summer Tour.pdf"' \
-F 'type="pdf"' \
-F 'messaging_product="whatsapp"'

{"id":"1237959477554785"}

In [19]:
# PYTHON
#media upload meta for dev
import os
import requests
from dotenv import load_dotenv



__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import sqlite3
import json

load_dotenv()


WHAT_TOKEN = "EAAOXhdmUSwoBOZBYUYhaRXNMTbNDoUnzSgjIMk36xZB0QZCbvote9sACCbSQuIRcDmEWlDJe2N5dZCxiY7f1cl6gbA3dTkmWOVEFHYZCtOwlbEWd2kiSeoTFBc3ZCmETIPdF9cPvT34rZBmSIjOwX6FeJxQvZC51EPPfXKLpD7cqiQlzsNpuIPnuCb9WMxD1X22vAruHVr9SoZCzIHA3KDJoZD"
VERSION = "v20.0"
PHONE_NUMBER_ID = "367811079754331"



import requests

headers = {
    'Authorization': f'Bearer {WHAT_TOKEN}',
}
f = open("00000104-Sedan Car Types.pdf", "rb")
files = {
    'file':(os.path.basename(f.name), f, 'application/pdf'),
    'type': (None, "pdf"),
    'messaging_product': (None, "whatsapp"),
}

response = requests.post('https://graph.facebook.com/v20.0/'+PHONE_NUMBER_ID+'/media', headers=headers, files=files)
r = response.text

json.loads(r)["id"]

'488856287215536'

In [ ]:
# FINAL CODE TO UPLOAD PDF FILE TO MEDIA AND SEND VIA WHATSAPP


# SEND pdf to whatsapp
from flask import Flask, jsonify, request, Response
import os
import requests
from dotenv import load_dotenv
import google.generativeai as genai
import json
import time


__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import sqlite3

load_dotenv()
app = Flask(__name__)


WHAT_TOKEN = "EAAOXhdmUSwoBO0xsTny64TmlGg5Gb4EyLMovKZCshCeeVtbx3tJPalK1HNiOAzZBkyvCUZArr4N57F1WNklfjMGAar5Fceo1RyJK9GZCLZCCSwdajQNaAlvnZAPtBP7xKQmZBzLxvRGaVeZBKxjZCAxFW3yZCPcl1tg5DsmfhxXwyHycLiZALZBQbcu12i8RpvrgSqCHNISIW0bHDeXU1zTq5ZCXl"
VERSION = "v20.0"
PHONE_NUMBER_ID = "367811079754331"
received_phone_num = "94769818269"

# send pdf to whatsapp
url_media = f"https://graph.facebook.com/{VERSION}/{PHONE_NUMBER_ID}/media"
url = f"https://graph.facebook.com/{VERSION}/{PHONE_NUMBER_ID}/messages"


def upload_file(file_path):
    # Define the headers for the request
    headers = {
        "Authorization": f"Bearer {WHAT_TOKEN}",
        "Content-Type": "application/json"
    }

    # Define the payload
    with open(file_path, "rb") as file:
        files = {
            "file": file
        }
        response = requests.post(url_media, headers=headers, files=files)

    if response.status_code == 200:
        media_id = response.json().get("media_id")
        print("File uploaded successfully:", media_id)
        return media_id
    else:
        print("Failed to upload file:", response.status_code, response.text)
        return None

def send_whatsapp_pdf(to, media_id):
    # Define the headers for the request
    headers = {
        "Authorization": f"Bearer {WHAT_TOKEN}",
        "Content-Type": "application/json"
    }

    # Define the payload to send the PDF
    data = {
        "to": to,
        "type": "document",
        "document": {
            "id": media_id,
            "caption": "Here is the PDF document you requested.", 
            "filename": "your-document-filename.pdf"

        }
    }

    # Make the POST request to send the document
    response = requests.post(url, headers=headers, json=data)

    # Check the response
    if response.status_code == 200:
        print("PDF sent successfully:", response.json())
    else:
        print("Failed to send PDF:", response.status_code, response.text)

# Example usage
pdf_file_path = "/home/senzmatepc27/Desktop/senzmate/Internal projects/Digital tourism/drive-download-20240708T053302Z-001/Inquiry Handling Flow/Inquiry Handling Flow.pdf"

# Upload the PDF file
media_id = upload_file(pdf_file_path)

# Send the PDF if the upload was successful
if media_id:
    send_whatsapp_pdf(received_phone_num, media_id)

In [4]:
#Quotation for driver 
#retrieve_information

from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
import json 

from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

user_input="I'm interested in visiting beaches and we are 10 people"
received_phone_num = "942455456"
load_dotenv()

GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
classes =["Question", "other"]
# Initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_tokens=250,
    temperature=0.5,
)
keys = {
            "days": 5,
            "pax": 10,
        }
    # Create prompt template
prompt = ChatPromptTemplate.from_messages(
        [
            ( "system",
                f"""
'You are a text-based Trip Plan Quotation Calculation ChatBot. Choose the trip plan cost based on the number of passengers and the number of days provided in the information'
                
Instructions:
- 'days': The number of days for which the service is required.
- 'pax': The number of passengers.
- 'kilometers': The total distance to be traveled.
- 'USD': Represents the pricing structure for the selected package, specifying the cost.

Pricing Structure:
- The pricing varies depending on the number of days and passengers. Use conditional statements to determine the price.
- Based on the number of days and passengers, choose the approximate price.

Number of Days 5 to 14:
- For each range of days (from 5 to 14), the price is different depending on the number of passengers.
- If there are 2 passengers, a lower rate is applied compared to when there are more passengers and You will be taking a 'CAR'.

Prices:
    If it Only 2 Passengers choose in here:
        if it's 5 days: USD 395 with 800 kilometer
        if it's 6 days: USD 435 with 850 kilometer
        if it's 7 days: USD 465 with 900 kilometer
        if it's 8 days: USD 485 with 900 kilometer
        if it's 9 days: USD 495 with 900 kilometer
        if it's 10 days: USD 545 with 1000 kilometer
        if it's 11 days: USD 595 with 1100 kilometer
        if it's 12 days: USD 635 with 1200 kilometer
        if it's 13 days: USD 675 with 1300 kilometer
        if it's 14 days: USD 715 with 1400 kilometer
        - 'You will be taking a CAR'

    If it More Than 2 Passengers choose in here:
        if it's 5 days: USD 495 with 800 kilometer
        if it's 6 days: USD 535 with 850 kilometer
        if it's 7 days: USD 565 with 900 kilometer
        if it's 8 days: USD 585 with 900 kilometer
        if it' 9 days: USD 595 with 900 kilometer
        if it's 10 days: USD 645 with 1000 kilometer
        if it's 11 days: USD 695 with 1100 kilometer
        if it's 12 days: USD 745 with 1200 kilometer
        if it's 13 days: USD 795 with 1300 kilometer
        if it's 14 days: USD 855 with 1400 kilometer
        - 'You will be taking a VAN'

Sample output format:
    *TOTAL PRICE: 395 USD - For 05 days, 02 Adults*

    _[Price given is for 800 kilometers]_

"""
                    
            ),
            ("human", "{information}"),
        ]
    )

# Create chain
chain = prompt | llm

# Generate the answer
ai_msg = chain.invoke({
    "information": keys
})

# Retrieve the results
result = ai_msg.content
print(result)

I0000 00:00:1723124434.474707  142012 tcp_posix.cc:809] IOMGR endpoint shutdown


*TOTAL PRICE: 495 USD - For 05 days, 10 Adults*

_[Price given is for 800 kilometers]_

_You will be taking a VAN_ 



I0000 00:00:1723124658.213793  142024 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1723124676.374285  142024 tcp_posix.cc:809] IOMGR endpoint shutdown


In [4]:
# Basic llm using context
#retrieve_information

from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory
import json 

from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# #Get the context related to the question from vector db
# def gen_context(vectordb, question):
#     docs = vectordb.similarity_search(question, k=3)
#     context = "\n".join([doc.page_content for doc in docs])
#     doc_list = [doc.metadata for doc in docs]
#     return context, doc_list

# question = "Can you recommend a trip plan to srilanka?"
question = "Can u tell the use of chatbot?"
# question = "What is chatbot?"


load_dotenv()

GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

# #Define paths
# persist_directory = '/home/senzmatepc27/Desktop/senzmate/Internal projects/Digital tourism/Github/Whatsapp-chatbot/docs/chroma_predef_trips'

# #Load Vector DB
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

# #Load context only relevant to the question
# context, doc_list = gen_context(vectordb, question)
# print("context:", context,"\n++++++++++++++")

context = """
A chatbot is an artificial intelligence program designed to simulate conversation with human users, often through messaging applications or \
websites. By leveraging natural language processing (NLP) and machine learning algorithms, chatbots can understand and respond to user \
inquiries in real-time, providing assistance with tasks such as customer support, information retrieval, and transaction facilitation. \
They can operate 24/7, offering instant responses and improving user engagement while reducing the workload on human agents. With advancements in technology, \
modern chatbots can maintain context, learn from interactions, and even adapt their tone to match user preferences, creating a more personalized experience. \
As businesses increasingly adopt chatbots, they are transforming the way companies interact with customers, enhancing efficiency and satisfaction in the digital landscape.
"""
    
#  Initialize the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    max_tokens=250,
    temperature=0.7,
)

    # Create prompt template
prompt = ChatPromptTemplate.from_messages(
        [
            ( "system",
                f"""
You are an information retrieval ChatBot. Answer the questions from the context only.

Instructions:
- Respond politely that I'm not able to answer your question if you couldn't find the answer from context.

"""
                    
            ),
            ("human", "{key_str}"),
        ]
    )

# Create chain
chain = prompt | llm

key = {
    "question" : question,
    "context" : context
}

key_str = json.dumps(key)
# Generate the answer
ai_msg = chain.invoke(key_str)

# Retrieve the results
result = ai_msg.content
print(result)

I0000 00:00:1723624572.003959  143016 tcp_posix.cc:809] IOMGR endpoint shutdown


Chatbots are used for customer support, information retrieval, and transaction facilitation. They can operate 24/7, offer instant responses and improve user engagement while reducing the workload on human agents. 



I0000 00:00:1723624805.754022  143020 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1723624814.081310  143020 tcp_posix.cc:809] IOMGR endpoint shutdown


In [7]:
import os
import json
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from dotenv import load_dotenv

class GeminiInitializer:
    def __init__(self):
        load_dotenv()
        self.api_key = os.environ.get('GOOGLE_API_KEY')
        genai.configure(api_key=self.api_key)
        print("Initializing the model")

    def run_text_model(self,
        prompt: str,
        model_name: str = "gemini-1.0-pro",
        temperature: float = 0
        ):
        safety_settings={
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE
        }
        llm = ChatGoogleGenerativeAI(model=model_name, temperature=temperature, safety_settings=safety_settings)
        response = llm.invoke(prompt)
        return response.content

    def extract_entities_relationships(self, prompt, model_name):
        try:
            res = self.run_text_model(prompt=prompt, model_name=model_name, temperature=0)
            return res
        except Exception as e:
            print(e)
            return None

g = GeminiInitializer()
g.run_text_model("response: hi")

Initializing the model


I0000 00:00:1723625563.531327  143015 tcp_posix.cc:809] IOMGR endpoint shutdown


'Hello! How can I help you today?'

I0000 00:00:1723625806.005472  143020 tcp_posix.cc:809] IOMGR endpoint shutdown


In [13]:
import pandas as pd
from SQL_IUD import get_user_records, update_record

def track_flow(id):
    index = id - 1
    # Step 1: Read the CSV file
    csv_file_path = '/home/senzmatepc27/Desktop/senzmate/Internal projects/Digital tourism/Github/Whatsapp-chatbot/chat_flow.csv'
    data = pd.read_csv(csv_file_path)

    data_dict = data.to_dict(orient='records')
    print(data_dict) 

    question = data_dict[index]["question"]
    return question

columns = "q_to_ask"
id = get_user_records(received_phone_num, columns)[0]["id"]
response = track_flow(id)

next_q = id + 1
query = """
UPDATE user_inputs
SET q_to_ask = {id}
WHERE Phone_num = {received_phone_num};
"""

update_record(query)




[{'id': 1, 'context': 'Greeting', 'question': 'Hello! How can I help you plan your trip today?', 'validation': nan, 'pharaphrase': nan}, {'id': 2, 'context': 'Basic Conversation', 'question': 'Which month will you be visiting Sri Lanka? How many days are you planning for your tour?', 'validation': nan, 'pharaphrase': nan}, {'id': 3, 'context': 'Interest Assessment', 'question': 'What activities or sights are you interested in? (You can choose multiple: Beaches, Wildlife, Culture, Mountains, Historical Places, Scenic Train Ride)', 'validation': nan, 'pharaphrase': nan}, {'id': 4, 'context': 'Itinerary Planning', 'question': 'itineraryAPI', 'validation': nan, 'pharaphrase': nan}, {'id': 5, 'context': 'Service Options', 'question': 'Would you like to have driver services only, or both driver and hotel bookings?', 'validation': nan, 'pharaphrase': nan}, {'id': 6, 'context': 'Retrieve More Information', 'question': 'May I know how many travelers will be joining?', 'validation': nan, 'pharap

In [25]:
#send request to flask app
import requests
import json

url = 'https://5dvbs9dn-5000.inc1.devtunnels.ms:5000/webhook'
myobj = {
   "object":"whatsapp_business_account",
   "entry":[
      {
         "id":"357268447472985",
         "changes":[
            {
               "value":{
                  "messaging_product":"whatsapp",
                  "metadata":{
                     "display_phone_number":"15556123988",
                     "phone_number_id":"373342025859932"
                  },
                  "contacts":[
                     {
                        "profile":{
                           "name":"Gowshi"
                        },
                        "wa_id":"94769818269"
                     }
                  ],
                  "messages":[
                     {
                        "from":"94769818269",
                        "id":"wamid.HBgLOTQ3Njk4MTgyNjkVAgASGCBCNTU1RTg1RkEzRUQwQzZCMTdEQjM0NEMzOEY0NDJGMAA=",
                        "timestamp":"1721111470",
                        "text":{
                           "body":"What are the tourist places to visit in srilanka?"
                        },
                        "type":"text"
                     }
                  ]
               },
               "field":"messages"
            }
         ]
      }
   ]
}
r = json.dumps(myobj)
print(type(r))

x = requests.post(url, json = r)

print(x.text)

<class 'str'>



In [17]:
import traceback

try:
    # Code that may raise an exception
    result = 10 / 0
except Exception as e:
    # Extract the traceback information
    tb = traceback.format_exc()
    
    # Print the exception message
    print(f"An error occurred: {e}")
    
    # Print the traceback information
    print("Traceback information:")
    print(tb)

An error occurred: division by zero
Traceback information:
Traceback (most recent call last):
  File "/tmp/ipykernel_6336/495492029.py", line 5, in <module>
    result = 10 / 0
             ~~~^~~
ZeroDivisionError: division by zero



In [28]:
# Define the timestamps
timestamp1 = 1724129252
timestamp2 = 1724218075
 
if timestamp1:
    print("hi")
# Determine which is latest
a =  timestamp1 > timestamp2
a

if not a:
    print("sdf")

hi
sdf


In [36]:
import re

# Sample string containing the SQL query
text = '''
SELECT (SELECT "2Pax" FROM Transport_table WHERE days = 6 AND Season = "Winter") AS "driver_cost", (SELECT Kilometers FROM Transport_table WHERE Season = "Winter" AND days = 6) AS "km", (SELECT ExtraRate FROM transport_extra_vehicle WHERE Pax = 2) AS "extra_rate", (SELECT Vehicle FROM transport_extra_vehicle WHERE Pax = 2) AS "vehicle"
'''

# Regular expression to match the SQL query
pattern = r'\bSELECT\b.*?'

# Find the SQL query using the regular expression
match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)

# Extract and print the SQL query
if match:
    sql_query = match.group(0)
    print(sql_query)
else:
    print("No SQL query found.")

SELECT


In [40]:
import re
import json
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

# Retrieve the API key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Define the classes for intent analysis
classes = ["Name", "Email", "Address", "Num_of_days", "Num_of_adults", "Num_of_kids", "Month", "Start_date", "End_date", "Interests", "Service"]

def generate_response(prev_question, user_input, received_phone_num, current_date):
    print("WELCOME TO Q&A SECTION")

    # Initialize the LLM
    llm = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash",
        max_tokens=250,
        temperature=0.5,
    )

    prompt = ChatPromptTemplate.from_messages(
          [
              ( "system",
                    """

    - extract values from information such as classes, received_phone_num, prev_question, user_input, current_date

    Today's date is current_date. Understand the current the month, year and date to extract information.
    You are a information retrieval ChatBot. Retrieve the following information of classes from the user_input. 
    To understand the meaning of the user_input refer the previous_question.

    previous_question: prev_question

    Description for information are:
    - 'Name refers to the name of the user.',
    - 'Email refers to the email of the user and the input should be in the default email format.',
    - 'Address refers to the address of the user.',
    - 'Num_of_days refers to how many days users plan to spend on their holiday or trip, and the input should be in numbers.',
    - 'Num_of_adults refers to how many adults plan to stay in a hotel or room, and the input should be in numbers.',
    - 'Num_of_kids refers to how many kids plan to stay in a hotel or room, and the input should be in numbers.',
    - 'Month refers to which month users plan to enjoy their trip or holiday, and the input should be the name of the month.',
    - 'Start_date refers to the day the trip starts, and the input should be in the date format.',
    - 'End_date refers to the day the trip ends, and the input should be in the date format.',
    - 'Interests refers to the user's interests or preferences for the trip (e.g., beach, wildlife, nature, adventure).',
    - 'Service refers to the service customer wants either Driver service only or Driver and Hotel booking services'


    Current input:
    user_input

    Example conversation:
        previous_question: How many days are you planning for the trip?
        user_input: 10
        output: 
        UPDATE user_inputs
        SET Num_of_days = "10"
        WHERE Phone_num = 46546;

        previous_question: How many adults are joining your trip?
        user_input: it is 2.
        output: 
        UPDATE user_inputs
        SET Num_of_adults = "2"
        WHERE Phone_num = 46546;


    Output format should be a JSON: You will respond to the input by producing two outputs in a JSON format.
    {{"intent_of_prev_question": identify the intent of previous question and output the respective information, "query":"UPDATE user_inputs SET Name = 'John', Email = 'john@gmail.com', Address = 'Swiss', Num_of_days = '7', Num_of_adults = '10', Num_of_kids = '2', Month = 'june', Start_date = '2024/10/10', End_date = '2024/10/20', Interests = 'nature and beaches', Service = 'Driver and hotel booking' WHERE Phone_num = received_phone_num;"}}

    Instructions:
    - Consider the previous_question and user_input while retrieving the information.
    - In the output only change the line starts with SET.
    - The output should be strictly in given format and avoid unnecessary words.
    - If the keys in line SET doesn't have values please remove those keys.
    - If all of the keys doesn't have values just output SELECT 1 FROM user_inputs;
    - If any service found in the user_input only please classify it in to one of the two categories such as Driver, Driver and Hotel. Else send SELECT 1 FROM user_inputs; as response and ask the question again to retrieve service information. 
    - If you find number of kids and it is in text format, Convert it into number. Example: "No kids" means 0.
    - Take user inputs as integers when you receive Num_of_days, Num_of_adults, and Num_of_kids.



    """
              ),
              ("human", "{information}"),
          ]
    )

    # Create chain
    chain = prompt | llm

    keys = {
        "user_input": user_input,
        "prev_question": prev_question,
        "received_phone_num": received_phone_num,
        "classes": f"{', '.join(classes)}",
        "current_date": current_date
    }
    keys_str = json.dumps(keys)

    # Generate the answer
    ai_msg = chain.invoke({
        "information": keys_str
    })

    # Retrieve the results
    result = ai_msg.content

    return result

prev_question = "How many adults are you planning for the trip?"
user_input = "gh"
received_phone_num = "0771234567"
current_date = "2024-08-23"

predict = generate_response(prev_question, user_input, received_phone_num, current_date)
# print(predict)

# # Regular expression to find JSON block
# json_pattern = re.compile(r'```json\n(.*?)\n```', re.DOTALL)

# # Search for the JSON block
# match = json_pattern.search(predict)
# # print(type[match])

# # File path
# file_path = 'C:/Users/user/Documents/VS/WhatsappChatbot/Whatsapp-chatbot/input.json'

# if match:
#     json_string = match.group(1)
    
#     # Convert JSON string to Python dictionary
#     data = json.loads(json_string)
    
# #     # Write dictionary to JSON file
# #     with open(file_path, 'w') as json_file:
# #         json.dump(data, json_file, indent=4)
    
# #     print(f"JSON data has been written to {file_path}")
# # else:
# #     print("No JSON block found")

# # Read JSON file and convert to dictionary
# # with open(file_path, 'r') as json_file:
# #     data = json.load(json_file)

def parse_output_new(text, pattern):
    import re

    sql_query = ''

    # Regular expression to match the SQL query

    # Find the SQL query using the regular expression
    match = re.search(pattern, text, re.IGNORECASE | re.DOTALL)
    print(match)

    # Extract and print the SQL query
    if match:
        sql_query = match.group(1)
        print(sql_query)
    else:
        print("No SQL query found.")

    return sql_query

pattern = r'```json\n(.*?)\n```'
data = parse_output_new(predict, pattern )
print("\n",data)

WELCOME TO Q&A SECTION


I0000 00:00:1724650554.777682  835832 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1724650555.176037  835831 tcp_posix.cc:809] IOMGR endpoint shutdown


<re.Match object; span=(0, 124), match='```json\n{"intent_of_prev_question": "Num_of_adul>
{"intent_of_prev_question": "Num_of_adults", "query": "UPDATE user_inputs SET  WHERE Phone_num = '0771234567';"}

 {"intent_of_prev_question": "Num_of_adults", "query": "UPDATE user_inputs SET  WHERE Phone_num = '0771234567';"}


I0000 00:00:1724650800.815224  835865 tcp_posix.cc:809] IOMGR endpoint shutdown


In [71]:
import dns.resolver

def has_mx_record(domain):
    try:
        # Query MX records
        records = dns.resolver.resolve(domain, 'MX')
        return len(records) > 0
    except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN):
        return False

def is_valid_email(email):
    try:
        local_part, domain = email.rsplit('@', 1)
        if not has_mx_record(domain):
            return False
        return True
    except ValueError:
        return False

# Example usage
email = "af@df.com"
print(is_valid_email(email))  # Output: True or False
print(has_mx_record(email))  # Output: True or False



True
False


[{"type": "human", "content": "Clear"}]
summary chat: The user said "Clear". 



In [6]:
result {
 "result": "That's fantastic! You'll have a great time experiencing everything Sri Lanka has to offer.  [Based on your interests here's the suggested itinerary for your trip. Does this look good to you?", "itineraryAPI"]",
 "source": "Acknowledging the user's interests and presenting the itinerary."
}

The dictionary has empty values for the following keys: ['email', 'address']
